# The structure of GPflowOpt
*Joachim van der Herten*

In this document, the structure of the GPflowOpt library is explained, including some small examples. First the `Domain` and `Optimizer` interfaces are shortly illustrated, followed by a description of the `BayesianOptimizer`. At the end, a step-by-step walkthrough of the `BayesianOptimizer` is given.

## Optimization
The fundamtental design principle of GPflowOpt is solving optimization problems of the form
$$\underset{\boldsymbol{x} \in \mathcal{X}}{\operatorname{argmin}} f(\boldsymbol{x}).$$ The *objective function* $f: \mathcal{X} \rightarrow \mathbb{R}^p$ maps a candidate optimum to a score (or multiple). Here $\mathcal{X}$ represents the input domain. This domain encloses all solutions to the optimization problem and can be entirely continuous (i.e., a $d$-dimensional hypercube) but may also consist of discrete and categorical parameters. 

In GPflowOpt, solving an optimization problem starts with defining the domain. This is achieved by combining parameters. This is how a simple square domain is defined:

In [1]:
from GPflowOpt.domain import ContinuousParameter
domain = ContinuousParameter('x1', -2, 2) + ContinuousParameter('x2', -1, 2)
domain

Name,Type,Values
x1,Continuous,[-2. 2.]
x2,Continuous,[-1. 2.]


Based on this domain, we can now easily apply one of the included optimizers to optimize objective functions. GPflowOpt defines an intuitive `Optimizer` interface which can be used to specify the domain, the initial point(s), constraints (to be implemented) etc.
Here is how a simple quadratic function is optimized using one of the available methods of SciPy's minimize:

In [2]:
import numpy as np
from GPflowOpt.optim import SciPyOptimizer

def fx(X):
    X = np.atleast_2d(X)
    # Return objective & gradient
    return np.sum(np.square(X), axis=1), 2*X


optimizer = SciPyOptimizer(domain, method='SLSQP')
optimizer.set_initial([-1,-1])
optimizer.optimize(fx)

     fun: 0.0
     jac: array([ 0.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 3
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([[ 0.,  0.]])

The objective function should return both objectives, as well as gradients. Some methods are inherently gradient-free (like Monte-Carlo optimization) and automatically discard the second returned array. 

In [3]:
from GPflowOpt.optim import MCOptimizer
optimizer = MCOptimizer(domain, 200)
optimizer.optimize(fx)

     fun: array([[ 0.00402339]])
 message: 'OK'
    nfev: 201
 success: True
       x: array([[-0.05970573,  0.0214152 ]])

## Bayesian Optimization

In Bayesian Optimization, the typical assumption is that $f$ is expensive to evaluate and no gradients are available. Hence the typical approach is to sequentially select a limited set of decisions $\boldsymbol{x}_0, \boldsymbol{x}_1, ... \boldsymbol{x}_{n-1}$ using a sampling policy. Hence each decision $\boldsymbol{x}_i \in \mathcal{X}$ itself is the result of an optimization problem
$$\boldsymbol{x}_i = \underset{\boldsymbol{x}}{\operatorname{argmax}} \alpha_i(\boldsymbol{x})$$

Each iteration, a function $\alpha_i$ which is cheap-to-evaluate acts as a surrogate for the expensive function. It is typically a mapping of the predictive distribution of a (Bayesian) model built on all decisions and their corresponding (noisy) evaluations. The mapping introduces an order in $\mathcal{X}$ to obtain a certain goal. The typical goal within the context of Bayesian Optimization is the search for *optimality* or *feasibility* while keeping the amount of required evaluations ($n$) a small number. As we can have several functions $f$ representing objectives and constraints, Bayesian Optimization may invoke several models and mappings $\alpha$. These mappings are typically referred to as *acquisition functions* (or *infill criteria*).

In GPflowOpt this typical flow itself is implemented as follows:

* Several acquisitions are available

* The optimization flow is implemented in its own optimizer which complies with the `Optimizer` interface.

Here the previous example is optimized using Bayesian Optimization instead, using the Expected Improvement acquisition function:

In [4]:
from GPflowOpt.bo import BayesianOptimizer
from GPflowOpt.design import FactorialDesign
from GPflowOpt.acquisition import ExpectedImprovement
import GPflow

# The Bayesian Optimizer does not expect gradients to be returned
def fx(X):
    X = np.atleast_2d(X)
    # Return objective & gradient
    return np.sum(np.square(X), axis=1)[:,None]

    
X = FactorialDesign(2, domain).generate()
Y = fx(X)

# initializing quite a standard BO model, Gaussian Process Regression with
# Matern52 ARD Kernel
model = GPflow.gpr.GPR(X,Y,GPflow.kernels.Matern52(2, ARD=True))
alpha = ExpectedImprovement(model)

# Now we must specify an optimization algorithm to optimize the acquisition 
# function, every iteration. 
acqopt = SciPyOptimizer(domain, method='SLSQP')

# Now create the Bayesian Optimizer
optimizer = BayesianOptimizer(domain, alpha, optimizer=acqopt)
optimizer.optimize(fx, n_iter=10)

/home/javdrher/.virtualenvs/gpflowopt/lib/python3.5/site-packages/GPflow/transforms.py:129: RuntimeWarning: overflow encountered in exp
  result = np.log(1. + np.exp(x)) + self._lower
/home/javdrher/.virtualenvs/gpflowopt/lib/python3.5/site-packages/GPflow/transforms.py:140: RuntimeWarning: overflow encountered in exp
  result = np.log(np.exp(y - self._lower) - np.ones(1, np_float_type))
/home/javdrher/.virtualenvs/gpflowopt/lib/python3.5/site-packages/GPflow/transforms.py:140: RuntimeWarning: divide by zero encountered in log
  result = np.log(np.exp(y - self._lower) - np.ones(1, np_float_type))


     fun: array([  4.19905705e-05])
 message: 'OK'
    nfev: 10
 success: True
       x: array([[  4.11478891e-09,  -6.48001316e-03]])

This brief snippet code starts from a 2-level grid (square corner points) and uses a GP model to model the response surface of the objective function. An acquisition function is specified. The `BayesianOptimizer` follows the same interface as the other optimizers and is initialized with a domain, the acquisition function and an additional optimization method to optimize the acquisition function each iteration. Finally, it runs for 10 iterations to optimize fx.

The code to evaluate the acquisition function on the model is written in TensorFlow, allowing gradient-based optimization without additional effort due to the automated differentation.

## Step-by-step description of the BayesianOptimizer